To calculate FST, I need to use unfolded SAF and SFS using the reference to polarize the alleles. I need a saf file for each population. first make filelists for each pop:


In [ ]:
 find . -name "*_pairedplus_marked_dupes.bam" -exec cp -i {} ../allBam/ \; 

In [ ]:
sed 's/$/_pairedplus_marked_dupes.bam/g' /local/home/sarahgw/WGS/Sequences/NJ_E/NJ_E_list>NJ_E_filelist

Calculate unfolded SAF for each population, see https://github.com/ANGSD/angsd/issues/65 for justification of folded vs unfolded

In [ ]:
for POP in NJ_E NJ_U NY_E NY_U VT_E VT_U; do echo $POP; ~/angsd/angsd -P 10 -b /local/home/sarahgw/WGS/Sequences/filelists/"$POP".filelist -anc /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -GL 1 -dosaf 1 -minMapQ 30 -minQ 20 -out /local/home/sarahgw/WGS/SNPS/All_mylu/"$POP"_un; done

Calculate 2d sfs for each pair 

In [ ]:
~/angsd/angsd/misc/realSFS -P 20 ~/WGS/saf/NJ_E_un.saf.idx ~/WGS/saf/NJ_U_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/NJ_EU_unfull

In [ ]:
~/angsd/angsd/misc/realSFS -P 10 ~/WGS/saf/NY_E_un.saf.idx ~/WGS/saf/NY_U_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/NY_EU_unfull

In [ ]:
~/angsd/angsd/misc/realSFS -P 10 ~/WGS/saf/VT_E_un.saf.idx ~/WGS/saf/VT_U_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/VT_EU_unfull.ml

In [ ]:
~/angsd/angsd/misc/realSFS -P 10 ~/WGS/saf/NJ_E_un.saf.idx ~/WGS/saf/NY_E_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/NJNY_E_unfull

In [ ]:
~/angsd/angsd/misc/realSFS -P 10 ~/WGS/saf/VT_E_un.saf.idx ~/WGS/saf/NY_E_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/VTNY_E_unfull

In [ ]:
~/angsd/angsd/misc/realSFS -P 10 ~/WGS/saf/NJ_E_un.saf.idx ~/WGS/saf/VT_E_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/NJVT_E_unfull

In [ ]:
~/angsd/angsd/misc/realSFS -P 10 ~/WGS/saf/NJ_E_un.saf.idx ~/WGS/saf/NY_E_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/NJNY_E_unfull


2) Use realSFS to calculate 2d sfs for each pair

3) Use the above calculated 2dsfs as priors jointly with all safs from step1 to calculate fst binary files

4) Use realSFS to extract the the fst values from the fst

So, I need the bam files sorted into populations and make a bam file list for each pop. Move them over to annotate using this

run sfs for each group on annotate

Restart full at 12:15 PM 7/7/17 (nothing else running on annotate) finished Sat Jul  8 00:22:05 EDT 2017 so it took 12 hours
Doing unfolded SFS 7/14/17 1 pm

In [ ]:
~/angsd/angsd/misc/realSFS -P 20 NY_E_un.saf.idx NY_U_un.saf.idx > NY_EU_unfull

Started on annotate at 1:42 PM on 7/11/17

In [ ]:
~/angsd/angsd/misc/realSFS -P 15 VT_E_un.saf.idx VT_U_un.saf.idx > VT_EU_unfull.ml; echo "VT SFS Done" | mailx gw.sarah@gmail.com

Do paired SFS for Exposed and unexposed populations

In [ ]:
~/angsd/angsd/misc/realSFS -P 15 NJ_U.saf.idx VT_U.saf.idx > NJVT_U.ml

In [ ]:
~/angsd/angsd/misc/realSFS -P 15 NJ_U.saf.idx NY_U.saf.idx> NJNY_U.ml

In [ ]:
~/angsd/angsd/misc/realSFS -P 15 VT_U.saf.idx NY_U.saf.idx> VTNY_U.ml

In [ ]:
~/angsd/angsd/misc/realSFS fst index NJ_U.saf.idx VT_U.saf.idx NY_U.saf.idx -sfs NJVTNY_U.ml -fstout NJVTNY_U

In [ ]:
~/angsd/angsd/misc/realSFS -P 15 NJ_E.saf.idx VT_E.saf.idx NY_E.saf.idx> NJVTNY_E.ml

Calculate fst for NJ

Running on Annotate started 7/9/17 at 9 AM finished o/n that evening

In [ ]:
~/angsd/angsd/misc/realSFS fst index NJ_E_un.saf.idx NJ_U_un.saf.idx -sfs NJ_EU_unfull.ml -fstout NJ_EU_unfull

In [ ]:
~/angsd/angsd/misc/realSFS fst stats NJ_EU_full.fst.idx

0.037613 0.053548

unfolded: > FST.Unweight[nObs:1718436087]:0.037613 Fst.Weight:0.053548.. well then, I guess it doesnt make a difference if I unfold the sfs?

In [ ]:
~/angsd/misc/realSFS fst stats2  NJ_EU_unfull.fst.idx -win 10000 -step 2000 -type 1 > NJ_unfull_sw2

In [ ]:
awk '$5>0.10' /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_sw 

Nevermind, it doesn't it outputs the numerator and denominator for FST. So, can caluclate FST for an individual SNP by dividing column 3 by column 4. 

In [ ]:
~/angsd/misc/realSFS fst print NJ_EU_unfull.fst.idx> NJ_EU_full_fst

In [ ]:
awk -F, 'FNR==NR {a[$1]=$0; next}; $1 in a {print a[$1]}' NJ_EU_full_FST NJ_SNP_sites1_sorted.txt

In [ ]:
~/angsd/misc/realSFS fst print VT_EU_full.fst.idx  > VT_EU_full_SNP_freq

In [ ]:
awk 'FNR==NR {a[$1$2]=$0; next}; $1$2 in a {print a[$1$2]}' VT_EU_full_SNP_freq VT_SNP_sites1_sorted.txt> VT_EU_full_SNP_fst

In [ ]:
~/angsd/misc/realSFS fst print NJ_EU_full.fst.idx > NJ_EU_full_SNP_freq

This seems to be arbitrarily filtering, the number of lines doesnt match and there are definitely sites in the output that aren't in the SNPs file, try again with a sorted SNP file? did't work. use awk to fiter by snps

In [ ]:
~/angsd/misc/realSFS fst print NJ_EU_full.fst.idx -sites NJ_SNP_sites1_sorted.txt > NJ_EU_full_SNP_freq1

In [ ]:
~/angsd/misc/realSFS fst print NY_EU_unfull.fst.idx -sites NY_SNP_sites1.txt > NY_EU_unfull_SNP_freq

In [ ]:
~/angsd/misc/realSFS fst print NY_EU_unfull.fst.idx> NJ_EU_unfull_fst

In [ ]:
~/angsd/misc/realSFS fst print VT_EU_full.fst.idx> VT_EU_full_fst

In [ ]:
grep 'AAPE02072785' NJ_EU_full_fst > NJ_AAPE02072785

This doesnt make much sense: mt change in allele freq: -0.0103409
full change in allele freq: -0.0153759

In [ ]:
awk '$4>0.40' /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_EU_full.fst.gz

Calculate FST for NY

Started at 1:15 PM on 7/11/17 on annotate

In [ ]:
~/angsd/angsd/misc/realSFS fst index NY_E_un.saf.idx NY_U_un.saf.idx -sfs NY_EU_unfull.ml -fstout NY_EU_unfull; echo "NY FST Done" | mailx gw.sarah@gmail.com

In [ ]:
~/angsd/angsd/misc/realSFS fst stats NY_EU_unfull.fst.idx 

This is the unfolded FST values: -> FST.Unweight[nObs:1710526195]:0.031527 Fst.Weight:0.051177
This is the folded FST values:-> FST.Unweight[nObs:1710526195]:0.030941 Fst.Weight:0.050585

In [ ]:
~/angsd/misc/realSFS fst stats2 NY_EU_unfull.fst.idx -win 10000 -step 2000 -type 1 > NY_unfull_sw2

Calculate FST for VT

In [ ]:
~/angsd/angsd/misc/realSFS fst index VT_E.saf.idx VT_U.saf.idx -sfs VT_EU_full.ml -fstout VT_EU_full ; echo "VT FST Done"| mailx gw.sarah@gmail.com 

In [ ]:
~/angsd/angsd/misc/realSFS fst stats VT_EU_full.fst.idx 

-> FST.Unweight[nObs:1734759399]:0.029444 Fst.Weight:0.045089

In [ ]:
~/angsd/misc/realSFS fst stats2 VT_EU_full.fst.idx -win 10000 -step 2000 -type 1 > VT_full_sw2

In [ ]:
awk '$5>0.10' /local/home/sarahgw/WGS/SNPS/All_mylu/VT_sw 

Get sequences for AAPE02072785

In [ ]:
grep 'AAPE02072785' -A 300 /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa | tr -d "\n\r" | cut -c 7000-15000 >/local/home/sarahgw/WGS/SNPS/All_mylu/AAPE02072785

In [ ]:
52357001 52365001

In [ ]:
grep 'GL429767' -A 300000 /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa | tr -d "\n\r" | cut -c 52357001-52365001 >/local/home/sarahgw/WGS/SNPS/All_mylu/GL429767

Test for why GL429767 is no longer an FST outlier? 
Can I pull out the allele frequencies and rerun FST analysis with new allele freqs but only subset of samples?
Can I look to see if genotype likelihoods or allele frquencies change in this region with full dataset?
Can I test if the change is on the part of the Exposed or Unexposed populations?--first, try doing fst with saf of old nj exposed and new nj unexposed.  

In [ ]:
~/angsd/angsd/misc/realSFS -P 15 angsdout_SAF_pairedplus_filt_E.saf.idx NJ_U.saf.idx > NJ_EU_sub.ml

In [ ]:
~/angsd/angsd/misc/realSFS fst index angsdout_SAF_pairedplus_filt_E.saf.idx NJ_U.saf.idx -sfs NJ_EU_sub.ml -fstout NJ_EU_sub

In [ ]:
~/angsd/angsd/misc/realSFS fst stats NJ_EU_sub.fst.idx 

FST.Unweight[nObs:1622446386]:0.046976 Fst.Weight:0.080651
Okay, so it seems like its not the exposed that are making a difference its the unexposed?

In [ ]:
~/angsd/angsd/misc/realSFS fst stats2 NJ_EU_sub.fst.idx -win 10000 -step 2000 -type 1 >  NJ_EU_sub_sw2